# Building a decision tree from scracth 

In [88]:
#Okay using only pandas and numpy 
import pandas as pd
import numpy as np

### Lets take a toy dataset to get the idea of a decision tree 

In [89]:
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [90]:
training_data

[['Green', 3, 'Apple'],
 ['Yellow', 3, 'Apple'],
 ['Red', 1, 'Grape'],
 ['Yellow', 3, 'Lemon']]

    Lets calculate gini index first so that it tells us how impure the current distribution is. 
    A value close to 0 indicates that the data is relatively pure , 
    while data closer to >=0.5 indicates that the decision is impure (i.e:there is a mix of 2 or more classes).   

In [91]:
def Gini_Index(data):
    """
    Given some input data we calculate the percentage representation of each output class wrt the data 
    We always assume to last column is the class labels
    """
    df=pd.DataFrame(data)
    #p contains the percentage probabilites of each category of output label,
    p=df[len(df.columns)-1].value_counts(normalize=True).values
    #We return the values as computed by the gini index formula
    return 1-sum(list(map(lambda n:n**2,p)))

In [92]:
# The value closer to 0.5 indicates a good mixture in the distribution , i.e : it is impure
Gini_Index(training_data)

0.625

Now we need to find a way to split the data into child nodes of a root to eventually build a tree.The way we derive child nodes is based on a rule which we use to split the data at the node into 2 child such that we minimize impurity on each split.   

In [93]:
#To split the data we need rules
class Rule:
    #We construct a rule object based on the column and the attribute on which rule needs to be applied for a split. 
    def __init__(self,feature_col,feature_value):
        self.feature_col=feature_col
        self.feature_value=feature_value
    
    #Simple function to check rule for any input value
    def check_rule(self,sample):
        #print(sample,"  ",self.feature_col)
        if sample[self.feature_col]==self.feature_value:
            return True
        else:
            return False
    #A representation function to return a string description when we try to print the object
    def __repr__(self):
        return f"Check for rule on {self.feature_col} if value is equal to {self.feature_value}"

In [94]:
#Lets make a rule where we want to check if a sample matchs the color red in column number 0
r1=Rule(0,'Red')

In [95]:
# call the check_rule() off the r1 object while passing the sample parameter for which the rule must be checked
r1.check_rule(['Red',3,'Apple'])

True

In [96]:
# Just to check if its working lets try a sample that must return a false
r1.check_rule(['Green',3,'Apple'])

False

In [97]:
# Lets try another rule on column 1 to check if diameter equals 3
r2=Rule(1,3)

In [98]:
r2.check_rule(['Yellow',1,"Lemon"])

False

In [99]:
r2.check_rule(['Green',3,"Apple"])

True

Now that we have the rule setup we need to be able to split a set of rows based on a rule.

In [100]:
def find_split(data,rule):
    """
    Input:
        data : the set of rows that need to split
        rule : type:Rule object 
               rule which needs to be used to split the set of rows passed as data.
    Returns:
        Two lists
        true_child and false_child 
        All the rows that match the rule are put into true_child and everything not matching the rule is in false_child
    """
    true_child=[]
    false_child=[]
    for sample in data:
        if rule.check_rule(sample) == True:
            true_child.append(sample)
        else:
            false_child.append(sample)
            
    return true_child,false_child

In [101]:
# Lets use the rule we made previous,r1, to split the training data
left,right=find_split(training_data,r1)

In [102]:
left #All the rows that match the rule

[['Red', 1, 'Grape']]

In [103]:
right #All the rows that did not match the rule

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

In [104]:
# Similarly we can split on rule r2 also
left,right=find_split(training_data,r2)
print("Left(Matched the rule)\n",left)
print("Right(Did not match the rule)\n",right)

Left(Matched the rule)
 [['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]
Right(Did not match the rule)
 [['Red', 1, 'Grape']]


## How do we know which rule to split on?

In [105]:
#TO being with lets calculate the Ginni_Index AKA the level of impurity before we split
Gini_Index(training_data)

0.625

In [106]:
#Lets try any rule and see after split 
left,right=find_split(training_data,Rule(0,"Green"))

In [107]:
#The left side (Rule-True side) is completely pure , hence gini_index=0
print(left)
Gini_Index(left)

[['Green', 3, 'Apple']]


0.0

In [108]:
#All values that dont match the rule are sent to right side and even so it contains impurities
print(right)
Gini_Index(right)

[['Yellow', 3, 'Apple'], ['Red', 1, 'Grape'], ['Yellow', 3, 'Lemon']]


0.6666666666666667

In [109]:
#Same thing we repeat for other classes of column 0
left,right=find_split(training_data,Rule(0,"Red"))
print(left)
print("Gini Index Of left child node :",Gini_Index(left))
print(right)
print("Gini Index Of left right node :",Gini_Index(right))

[['Red', 1, 'Grape']]
Gini Index Of left child node : 0.0
[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]
Gini Index Of left right node : 0.4444444444444444


In [110]:
#Same thing we repeat for other classes of column 0
left,right=find_split(training_data,Rule(0,"Yellow"))
print(left)
print("Gini Index Of left child node :",Gini_Index(left))
print(right)
print("Gini Index Of left right node :",Gini_Index(right))

[['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]
Gini Index Of left child node : 0.5
[['Green', 3, 'Apple'], ['Red', 1, 'Grape']]
Gini Index Of left right node : 0.5


Gini index tells us how impure the data after split results in , we would like to have splits that produce 
 the purest possible partitions on the data and also we would like to have larger less impure splits instead of
 single values/very small partitions of completely pure data.
 Hence we use a weighted sum of the splits to determine the best rule for splitting,A.K.A Information Gain

In [111]:
def Info_Gain(left,right,original_gini_index):
    #Calculating the weights based on the size of the partition
    wt=[ len(left)/(len(left)+len(right)) ,  len(right)/(len(left)+len(right))]
    #Now we use the weighted sum to account for the above description
    
    #subtracting the weighed gini_indexs from current uncertainity(a.k.a gini_index(parent_node))
    return original_gini_index - (wt[0]*Gini_Index(left) + wt[1]*Gini_Index(right))

In [112]:
#Now lets see how much infomation we gain by splitting on the class in column 0 (color)
left,right=find_split(training_data,Rule(0,"Yellow"))
Info_Gain(left,right,Gini_Index(training_data))

0.125

In [113]:
left,right=find_split(training_data,Rule(0,"Red"))
Info_Gain(left,right,Gini_Index(training_data))

0.2916666666666667

In [114]:
left,right=find_split(training_data,Rule(0,"Green"))
Info_Gain(left,right,Gini_Index(training_data))

0.125

In [115]:
#Of all the rules we see that the info gain is maximum if we partiton the data with the rule as red for col 0

In [116]:
#Lets write a function such that it recieves the data and reutns the best rule to split the data on

In [117]:
def find_best_rule(data):
    """
    Input: 
        A list of rows
    Returns:
        best_rule -> A rule object generating the max info gain
        max_info_gain -> the info gain generated by splitting the data on the best_rule
    """
    max_info_gain=-1
    best_rule=None
    #For each column
    for col in range(len(data[0])-1):
        #For each unique category of the column
        for val in np.unique(np.array(data)[:,col]):
            feature_col=col
            feature_value=val
            #Try every rule ==> test_rule
            test_rule=Rule(feature_col,feature_value)
            #Split the data based on the test_rule
            left,right=find_split(data,test_rule)
            if len(left)==0 or len(right)==0:
                continue
            #calculate the information gain for split based on current test_rule
            info_gain=Info_Gain(left,right,Gini_Index(data))
            #print(f"Infogain of {col} and {val} is : {info_gain}")
            #If info_gain is the more than known max_info_gain, update
            if info_gain>max_info_gain:
                max_info_gain=info_gain
                best_rule=test_rule
    return best_rule,max_info_gain 
            

In [118]:
# Testing the best rule function
best_rule,max_gain= find_best_rule(training_data)

In [119]:
best_rule

Check for rule on 0 if value is equal to Red

In [120]:
max_gain

0.2916666666666667

In [121]:
#Now that we have the best rule we can split the training data

In [122]:
left_child,right_child=find_split(training_data,best_rule)

In [123]:
#Now we need to see if we can split the child nodes furthur to possibly improve the purity
find_best_rule(left_child)

(None, -1)

If the find_best_rule() function returns (None,-1) it means the node already produces a pure group and splitting isnt requried furthur.

In [124]:
find_best_rule(right_child)

(Check for rule on 0 if value is equal to Green, 0.1111111111111111)

## How we have all the tools we need to start building the tree

In [125]:
#Lets define a class Node for the tree

In [164]:
class Node():
    def __init__(self,data,rule=None):
        self.data=data
        self.rule=rule
        self.left=None #True branch
        self.right=None #Flase branch
        

In [128]:
#Construct Tree

In [165]:
ctr=0
def make_tree(training_data,curr_node,prev_score):
    """
    Input : 
            training_data : a list of rows for building the tree
            curr_node : Initially while calling this function create a root node of type <Node> and pass it here.
            prev_score : Initally defaults to 0. Through recursion values are pass in the process of bulding the tree.
    """
    
    global ctr
    
    # If we dont recieve any data then we just return 
    if len(training_data)==0 :
        return
    
    #If we do recieve data we store it in the node's data attribute
    curr_node.data=training_data
    
    #Then we run the best_rule function over this data 
    best_rule=find_best_rule(curr_node.data)
    
    #If the purest split is alreadt achieved then we wont have a best rule to split hence it will return None so we return
    if best_rule[0]==None:
        print(curr_node.data)
        return
    
    #Bunch of prints to understand whats happeneing
    print("datastarts","====="*ctr)
    print("This DATA :")
    [print(row) for row in curr_node.data]
    print("is SPLIT On the rule ",best_rule,"\n")
    print("dataends","====="*ctr)
    
    # If the parent node has better information gain comparent to splitting by a rule dont split 
    if prev_score<best_rule[1]:
        #If we get a rule which improves the info-gain we place it into the node
        curr_node.rule=best_rule
        print(curr_node.rule,"\n\n")
        #And split the data based on the best rule to constrcut and assign left and right child nodes
        left_child,right_child=find_split(training_data,curr_node.rule[0])
        curr_node.left=Node(left_child)
        curr_node.right=Node(right_child)
        print("Start Left sub tree")
        ctr+=1
        #Recursive function call to construct the left sub tree
        make_tree(curr_node.left.data,curr_node.left,curr_node.rule[1])
        print("Left sub tree done")
        ctr=1
        #Recursive function call to construct the right sub tree
        print("Start Right sub tree")
        make_tree(curr_node.right.data,curr_node.right,curr_node.rule[1])
        print("Right sub tree done")
        
    return

In [166]:
#First create a empty root node
root=Node([])
#Pass the training data along with the root and initial score to build the tree
make_tree(training_data,root,0)

datastarts 
On the rule  (Check for rule on 0 if value is equal to Red, 0.2916666666666667) we split the Current data:

['Green', 3, 'Apple']
['Yellow', 3, 'Apple']
['Red', 1, 'Grape']
['Yellow', 3, 'Lemon']
dataends 
(Check for rule on 0 if value is equal to Red, 0.2916666666666667) 


Start Left sub tree
[['Red', 1, 'Grape']]
Left sub tree done
Start Right sub tree
datastarts =====
On the rule  (Check for rule on 0 if value is equal to Green, 0.1111111111111111) we split the Current data:

['Green', 3, 'Apple']
['Yellow', 3, 'Apple']
['Yellow', 3, 'Lemon']
dataends =====
Right sub tree done


Now that we have constructed the tree all we need to do is traverse the tree for a test sample, the rule at each node is matched and the appropriate child is visited until we reach a leaf node which outputs the majority labels contained at its node.data as the output class for that leaf.

In [168]:
def traverse_tree(root,sample):
    #If we dont get a root node return
    if root==None:
        return
    #If we are at the leaf node , print out all the labels and their percentage representation at the leaf
    if root.left==None and root.right==None:#root.rule==None:
        res_classes=np.array(root.data)[:,-1]
        class_label,count=np.unique(res_classes,return_counts=True)
        [print(class_label[x]," : ",count[x]/sum(count)) for x in range(len(class_label)) ]
        return
    
    #If it is not a leaf node match the values against the rule at the node and take the appropriate path
    elif root.rule[0]!=None:
        print("Checked for :","\t",root.rule)
        if root.rule[0].check_rule(sample):
            root=root.left
        else:
            root=root.right
    
    #print(root)
    #print(root.left)
    #print(root.right)
    
    #Finally make a recursive call to keep traversing until a leaf node is reached
    traverse_tree(root,sample)

In [175]:
#Same as above , we construct the tree 
root=Node([])
make_tree(training_data,root,0)

In [176]:
# Traverse the tree 
####
# NOTE
# Though we have only 2 input features we need to pass the sample having an element in place of the class label
# Here ive left it as a ? , as it is ignored but the len() of the list matters during index slicing 
####
traverse_tree(root,["Black",1,"?"])

'Apple'

In [177]:
#Now lets try it with a little bigger dataset which looks like this
df=pd.read_csv('Datasets/id3.csv')
df

,Outlook,Temperature,Humidity,Wind,Answer
0,sunny,hot,high,weak,no
1,sunny,hot,high,strong,no
2,overcast,hot,high,weak,yes
3,rain,mild,high,weak,yes
4,rain,cool,normal,weak,yes
5,rain,cool,normal,strong,no
6,overcast,cool,normal,strong,yes
7,sunny,mild,high,weak,no
8,sunny,cool,normal,weak,yes
9,rain,mild,normal,weak,yes


In [178]:
#First lets convert this dataframe into a list of rows
data=df.values.tolist()

In [179]:
#Create a root node and construct the tree
root=Node([])
make_tree(data,root,0)

In [180]:
#traverse the tree for a test sample
traverse_tree(root,["sunny","hot","normal","strong",""])

'yes'

## Here all the logging and prints are removed for a cleaner output and now in the traverse tree function we only return the label with the most confidence/representation in the leaf node.

In [181]:
ctr=0
def make_tree(training_data,curr_node,prev_score):
    global ctr
    if len(training_data)==0 :
        return
    
    curr_node.data=training_data
    best_rule=find_best_rule(curr_node.data)
    if best_rule[0]==None:
        #print(curr_node.data)
        return
    #print("datastarts====="*ctr)
    #print("On the rule ",best_rule,"we split the Current data:\n")
    #[print(row) for row in curr_node.data]
    #print("dataends====="*ctr)
    
    if prev_score<best_rule[1]:
        curr_node.rule=best_rule
        #print(curr_node.rule,"\n\n")
        left_child,right_child=find_split(training_data,curr_node.rule[0])
        curr_node.left=Node(left_child)
        curr_node.right=Node(right_child)
        #print("Start Left sub tree")
        ctr+=1
        make_tree(curr_node.left.data,curr_node.left,curr_node.rule[1])
        #print("Left sub tree done")
        ctr=1
        #print("Start Right sub tree")
        make_tree(curr_node.right.data,curr_node.right,curr_node.rule[1])
        #print("Right sub tree done")
    return

In [182]:
def traverse_tree(root,sample):
    
    if root.left==None and root.right==None:#root.rule==None:
        res_classes=np.array(root.data)[:,-1]
        class_label,count=np.unique(res_classes,return_counts=True)
        #[print(class_label[x]," : ",count[x]/sum(count)) for x in range(len(class_label)) ]
        ind=np.argmax(count)
        #print(class_label[ind])  # prints the most frequent element
        return class_label[ind]
    
    elif root.rule[0]!=None:
        #print("Checked for :","\t",root.rule)
        if root.rule[0].check_rule(sample):
            root=root.left
        else:
            root=root.right
    
    #print(root)
    #print(root.left)
    #print(root.right)
    return traverse_tree(root,sample)
     

In [184]:
res=traverse_tree(root,["sunny","hot","normal","strong",""])
res

'yes'

## Now time to try this on a big dataset

In [185]:
adf=pd.read_csv('Datasets/adult.data.csv',header=None).sample(frac=1)

In [187]:
data=adf[[1,5,8,9,14]].values
data.shape #around 32k rows

(32561, 5)

In [188]:
train=data[:-3000]

In [189]:
test=data[-3000:]

In [190]:
#we follow the same procedure, create a root and then construct the tree 
root=Node([])
make_tree(train,root,0)

In [157]:
#Let us use a for loop to go over a set of test sample ,3000 rows,and we'll try to gauge the models accuracy on this set. 

#initialize the score to be 0
score=0
#Loop over each test sample and obtain the prediction according to the tree and then compare it with the true value 
for row in test:
    if row[-1]==traverse_tree(root,row):
        #if prediction is true then increment score
        score+=1

In [151]:
#Accuracy
score/3000

0.754

# End